In [204]:
import os
import os.path as op
import coinsmeg_data as coinsmeg
from glob import glob
import pathlib
import mne
import re
import pandas as pd
import numpy as np

In [205]:
############## ------- Directories ---------- ############

# Directories
data_dir = coinsmeg.RAW_DIR # this is the same as BASE_DIR as raw data is stored in the base directory
maxfiltered_dir = op.join(coinsmeg.DERIVATIVES_DIR, "maxfiltered") # the input
aligned_dir = op.join(coinsmeg.DERIVATIVES_DIR, "aligned") # the output

# Get subjects
sub_run_combo = 'sub-04_run-4'
subject_id, run_id = sub_run_combo.split('_') # will produce subject_id = 'sub-XX', run_id = 'run-X'
run_number = int(re.search(r'\d+', run_id).group()) # extract number after 'run-

In [206]:
fif_file = f'{maxfiltered_dir}/{subject_id}/{subject_id}_ses-2-meg_task-coinsmeg_{run_id}_meg_transsss.fif'
beh_file = coinsmeg.get_sub_behav_fpath(subject_id,run_number)

In [207]:
coinsmeg.EVENT_ID

{'blockEnd': 20,
 'blockStart': 10,
 'expEnd': 105,
 'expStart': 100,
 'keyDown': 6,
 'keyLeft': 4,
 'keyRelease': 7,
 'keyRight': 3,
 'keyUp': 5,
 'laserHit': 1,
 'laserMiss': 2}

In [208]:
valid_triggers = list(coinsmeg.EVENT_ID.values())
overlapTriggers = [value + 1 for value in valid_triggers] + [value + 2 for value in valid_triggers]

In [209]:
minLaserDist = 0.08

In [210]:
############ ============== REQUIRED FUNCTIONS ========================= ##################

def calculate_block_times(all_events):
    """
    For an EEG dataset in MNE-Python, determines the start and end times of blocks.
    
    Parameters:
    - all_events: EEG events extracted from MNE Raw object 

    Returns:
    - eeg_block_times: A dictionary with block start and end indices, durations in minutes, 
                       and start/end times in seconds.
    """
    trig_values = all_events[:, 2]  # Event trigger values

    # Identify Block Start and End Indices
    trig_blockStart = coinsmeg.EVENT_ID['blockStart']
    trig_blockEnd = coinsmeg.EVENT_ID['blockEnd']

    blockStart_idx = np.where(trig_values == trig_blockStart)[0]
    blockEnd_idx = np.where(trig_values == trig_blockEnd)[0]

    if len(blockStart_idx) != 4 or len(blockEnd_idx) != 4:
        raise ValueError("Wrong number of block triggers detected")
    else:
        print("THere are four block start and end triggers.")

    # Calculate Block Durations (in minutes)
    # Convert the sample times to actual time in seconds
    blockStart_times = raw.times[all_events[blockStart_idx, 0]]
    blockEnd_times = raw.times[all_events[blockEnd_idx, 0]]

    block_durations = (blockEnd_times - blockStart_times) / 60  # Convert seconds to minutes

    # Warn about Suspicious Block Durations
    if any(block_durations > 4) or any(block_durations < 3):
        print("Warning: Suspicious block durations")

    # Collect and Return Block Information
    eeg_block_times = {
        'eventIndices': np.column_stack((blockStart_idx, blockEnd_idx)),
        'durationsInMin': block_durations,
        'bordersInSecs': np.column_stack((blockStart_times, blockEnd_times))
    }

    return eeg_block_times


In [211]:
def extract_csv_events(blockData, block_i):
        
    """
    From a session's csv file, extract csv events for the specified block. 
    Note there are four blocks in every session csv file.
    """

    blockData_currBlock = blockData[blockData.blockID == block_i]
    nFrames = blockData_currBlock['currentFrame'].iloc[-1]# Get the last element of blockData['currentFrame']
    # convert timestamps in frames to time unit (seconds)
    fsample = 60
    csv_times = np.arange(0, nFrames / fsample, 1 / fsample)
    # find indices in triggerValue where the value is greater than 0 (i.e., a trigger was sent)
    csv_trigIndex_blockSpace = np.where(blockData_currBlock['triggerValue'] > 0)[0] # which row of framewise block data has trigger
    csv_trig_rows = blockData_currBlock.iloc[csv_trigIndex_blockSpace]

    csv_time = csv_times[csv_trigIndex_blockSpace]
    csv_values = csv_trig_rows.triggerValue

    csv_start_value = csv_values[0]
    if csv_start_value !=  coinsmeg.EVENT_ID['laserHit'] and csv_start_value != coinsmeg.EVENT_ID['laserMiss']:
        print('first trigger in csv file for the block is not a laser event')

    csv_trigIndex_trigSpace = np.arange(1, len(csv_values) + 1)

    return csv_times, csv_values, csv_start_value, csv_trigIndex_trigSpace, csv_trigIndex_blockSpace


In [212]:
def extract_eeg_and_align_start(raw, all_events, eeg_block_times, block_i, csv_start_value):
   
    #### ======== Extract EEG events for the specified block   ========== ####

    # Step 1: Extract EEG events for the specified block
    start_idx, end_idx = eeg_block_times['eventIndices'][block_i-1] # get the block_i-1 element of eventIndices; contains start and end time of n block
    eegEvents = all_events[start_idx:end_idx + 1]  # MNE events are indexed as [start:end+1] in Python

    # Extract sample times and event codes for the current block
    eeg_times = raw.times[eegEvents[:, 0]]  # Convert sample indices to times in seconds
    eeg_values = eegEvents[:, 2]            # Extract the event codes (values) for these events

    #### Align EEG time with csv times by setting the first matching event time to zero ####

    tmpIdx = np.where(eeg_values == csv_start_value)[0] # gets the indices of elements of eeg_values that match csv_start_value
    if len(tmpIdx) == 0:
        raise ValueError("No matching event found in EEG for the specified csv_start_value.")

    # Get the first event in the EEG file which equals the first event of csv file
    # and get the time (in eeg recording) when it happened
    eegStartTimeOffset = eeg_times[tmpIdx[0]] # this is the same thing as 'how much earlier did eeg recording start compared to first csv event'

    # Subtract the offset to align times, setting the first matching event to time zero
    eeg_times = eeg_times - eegStartTimeOffset

    # Step 3: Calculate sample indices for the start and end of the block
    # Find the sample indices that correspond to the start and end times of the block
    eegStartSample = raw.time_as_index(eeg_block_times['bordersInSecs'][block_i-1, 0])[0] # time_as_index coverts seconds to indices, accounting for srate 
    eegEndSample = raw.time_as_index(eeg_block_times['bordersInSecs'][block_i-1, 1])[0] # both lines return an array of a single element, but we need [0] anyway to extract the element

    return eeg_times, eeg_values, eegStartTimeOffset, eegStartSample, eegEndSample


In [213]:
# Load in EEG data and wrangle the events a bit

raw = mne.io.read_raw_fif(fif_file, preload=True)
all_events = mne.find_events(raw, min_duration=0.005)
#exclude_event_ids = [10, 20, 100, 105]

# Step 3: Filter to only include events not in the exclude list
#stim_events = all_events[~np.isin(all_events[:, 2], exclude_event_ids)]

Opening raw data file /ohba/pi/lhunt/datasets/coins-meg_data/derivatives/maxfiltered/sub-04/sub-04_ses-2-meg_task-coinsmeg_run-4_meg_transsss.fif...
    Range : 5000 ... 837999 =      5.000 ...   837.999 secs
Ready.
Reading 0 ... 832999  =      0.000 ...   832.999 secs...


/tmp/ipykernel_1402234/898605740.py:3: RuntimeWarning: This filename (/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/maxfiltered/sub-04/sub-04_ses-2-meg_task-coinsmeg_run-4_meg_transsss.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_file, preload=True)


2867 events found
Event IDs: [  1   2   3   4   5   6   7  10  20 100 105]


In [214]:
blockData = pd.read_csv(beh_file,usecols=range(12))
block_i = 1

In [215]:
csv_times, csv_values, csv_start_value, csv_trigIndex_trigSpace, csv_trigIndex_blockSpace = extract_csv_events(blockData, 1)

In [216]:
## Now the EEG triggers
eeg_block_times = calculate_block_times(all_events)

THere are four block start and end triggers.


In [217]:
eeg_times, eeg_values, eegStartTimeOffset, eegStartSample, eegEndSample = extract_eeg_and_align_start(raw, all_events, eeg_block_times, block_i, csv_start_value)

In [203]:
## Make a copy of everything!!
csv_times_original = csv_times.copy()
csv_values_original = csv_values.copy()

eeg_times_original = eeg_times.copy()
eeg_values_original = eeg_values.copy()

### Now the trickier aligning fun begins...

In [218]:
# Initialize assignments and tracking arrays
assignments = np.zeros(len(eeg_values), dtype=int)  # Track number of assignments per EEG event
assigned_csv_trigs = [[] for _ in range(len(eeg_values))]  # Track assigned csv triggers
unassigned_csv_trigs_idx = []  # Track unassigned csv triggers


In [219]:
# Find the indices of eeg_values that are not in csv_values (the csv file containing ground-truth trigger values)
toremove = ~np.isin(eeg_values, csv_values)

In [201]:
# Remove those indices from eeg_values and eeg_times
eeg_values = eeg_values[~toremove]
eeg_times = eeg_times[~toremove]

In [232]:
for csv_idx, csv_value in enumerate(csv_values):

    # make eeg_values the same length as csv values
    eeg_values2 = np.append(eeg_values, np.ones(len(csv_values) - len(eeg_values)) * np.nan)

    # Find potential EEG triggers with matching values
    potEEGtrigs = np.where(eeg_values2 == csv_values)[0]
    
    if len(potEEGtrigs) == 0:
        # No matching EEG value found for the csv event
        print("Warning: Unassigned MATLAB event because no matching EEG value found...")
        unassigned_csv_trigs_idx.append(csv_idx)
    else:
        # Find the closest EEG trigger by time
        time_diffs = np.abs(eeg_times[potEEGtrigs] - csv_times[csv_idx])
        idx = np.argmin(time_diffs)  # Index of the closest time match
        closest_eeg_idx = potEEGtrigs[idx]

        # Assign the csv trigger to this EEG trigger
        assignments[closest_eeg_idx] += 1

        # Check for multiple assignments to the same EEG trigger
        if assignments[closest_eeg_idx] > 1:
            # Double assignment detected - resolve based on timing

            # Times of the csv events in question
            t1 = csv_times[assigned_csv_trigs[closest_eeg_idx][0]]
            t2 = csv_times[csv_idx]

            # Check if the two csv events are very close in time
            if t2 - t1 < minLaserDist:
                flagMissedEEG = True
            else:
                flagMissedEEG = False

            # Calculate delays for each assignment
            delay1 = t1 - eeg_times[closest_eeg_idx]
            delay2 = t2 - eeg_times[closest_eeg_idx]

            # Estimate the current average delay
            try:
                indices2use = potEEGtrigs[max(0, idx - 10):idx]
            except IndexError:
                indices2use = potEEGtrigs[:idx]
            
            indices2use = indices2use[assignments[indices2use] > 0]  # Non-empty assignments
            if len(indices2use) > 0:
                currentDelay = np.mean(csv_times[np.array(assigned_csv_trigs)[indices2use]] - eeg_times[indices2use])
            else:
                currentDelay = 0

            # Decide which csv event fits better
            if np.sign(delay1) != np.sign(delay2):
                betterFit = 'first' if np.sign(currentDelay) == np.sign(delay1) else 'second'
            else:
                betterFit = 'first' if abs(delay1 - currentDelay) < abs(delay2 - currentDelay) else 'second'

            # Reassign based on the better fit
            if betterFit == 'first':
                assignments[closest_eeg_idx] = 1
                if not flagMissedEEG:
                    # Assign the next closest EEG trigger
                    next_idx = potEEGtrigs[idx + 1] if idx + 1 < len(potEEGtrigs) else None
                    if next_idx is not None:
                        assignments[next_idx] += 1
                        assigned_csv_trigs[next_idx].append(csv_idx)
                    else:
                        unassigned_csv_trigs_idx.append(csv_idx)
                else:
                    unassigned_csv_trigs_idx.append(csv_idx)
            else:
                # Replace the assignment with the current csv event
                prev_csv = assigned_csv_trigs[closest_eeg_idx][0]
                assigned_csv_trigs[closest_eeg_idx] = [csv_idx]
                if not flagMissedEEG:
                    # Try to reassign previous csv event to another EEG trigger
                    prev_idx = potEEGtrigs[idx - 1] if idx - 1 >= 0 else None
                    if prev_idx is not None and assignments[prev_idx] < 1:
                        assignments[prev_idx] += 1
                        assigned_csv_trigs[prev_idx].append(prev_csv)
                    else:
                        unassigned_csv_trigs_idx.append(prev_csv)
                else:
                    unassigned_csv_trigs_idx.append(prev_csv)
        else:
            # Standard assignment without double assignments
            assigned_csv_trigs[closest_eeg_idx].append(csv_idx)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (758,) + inhomogeneous part.